[Link to example](https://www.flexcompute.com/tidy3d/examples/notebooks/TunableChiralMetasurface/)

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import tidy3d as td
import tidy3d.web as web
from tidy3d.plugins.dispersion import AdvancedFastFitterParam, FastDispersionFitter
import scienceplots

# Runtime Analysis Postprocess
plt.style.use(['science', 'notebook', 'grid'])


In [2]:
lda0 = 2.5  # central wavelength
freq0 = td.C_0 / lda0  # central frequency
ldas = np.linspace(2, 3, 101)  # wavelength range
freqs = td.C_0 / ldas  # frequency range
fwidth = 0.5 * (np.max(freqs) - np.min(freqs))  # width of the source frequency range


In [6]:
Px = 1.25  # unit cell period in the x direction
Py = 0.4  # unit cell period in the y direction
A = 0.24  # width of the Al resonator
B = 0.21  # width of the slot
alpha = 30 * np.pi / 180  # tilted angle of the slot
t_gst = 0.075  # thickness of the GST layer
t_al2o3 = 0.025  # thickness of the Al2O3 capping layer
t_al = 0.06  # thickness of the Al layer

In [7]:
Al = td.material_library["Al"]["RakicLorentzDrude1998"]

coeffs = [(1.431, 0.0727**2), (0.651, 0.119**2), (5.341, 18.028**2)]
Al2O3 = td.Sellmeier(coeffs=coeffs)


In [9]:
inf_eff = 1e2  # effective infinity

# calculate the vertices of the resonator on the left
vertices = [
    (-inf_eff, A / 2),
    (-inf_eff, -A / 2),
    (-(B / np.cos(alpha) + A * np.tan(alpha)) / 2, -A / 2),
    (-(B / np.cos(alpha) - A * np.tan(alpha)) / 2, A / 2),
]

# define the left resonator structure
left_resonator = td.Structure(
    geometry=td.PolySlab(vertices=vertices, axis=2, slab_bounds=(0, t_al)), medium=Al
)

# calculate the vertices of the resonator on the right
vertices = [
    (inf_eff, A / 2),
    (inf_eff, -A / 2),
    ((B / np.cos(alpha) - A * np.tan(alpha)) / 2, -A / 2),
    ((B / np.cos(alpha) + A * np.tan(alpha)) / 2, A / 2),
]

# define the right resonator structure
right_resonator = td.Structure(
    geometry=td.PolySlab(vertices=vertices, axis=2, slab_bounds=(0, t_al)), medium=Al
)

# define the Al2O3 buffer layer
buffer_layer = td.Structure(
    geometry=td.Box(center=(0, 0, -t_al2o3 / 2), size=(td.inf, td.inf, t_al2o3)), medium=Al2O3
)


# define a function to construct the GST layer given the state
def make_gst_layer(state):
    if state == "amorphous":
        medium = GST_A
    elif state == "crystalline":
        medium = GST_C
    else:
        raise ValueError("state must be `amorphous` or `crystalline`")

    gst_layer = td.Structure(
        geometry=td.Box(center=(0, 0, -t_al2o3 - t_gst / 2), size=(td.inf, td.inf, t_gst)),
        medium=medium,
    )
    return gst_layer


# define the Al substrate
al_substrate = td.Structure(
    geometry=td.Box.from_bounds(
        rmin=(-inf_eff, -inf_eff, -inf_eff), rmax=(inf_eff, inf_eff, -t_al2o3 - t_gst)
    ),
    medium=Al,
)


In [10]:
def circular_polarized_plane_wave(pol):
    # define a plane wave polarized in the x direction
    plane_wave_x = td.PlaneWave(
        source_time=td.GaussianPulse(freq0=freq0, fwidth=fwidth),
        size=(td.inf, td.inf, 0),
        center=(0, 0, 0.3 * lda0),
        direction="-",
        pol_angle=0,
    )

    # determine the phase difference given the polarization
    if pol == "left":
        phase = -np.pi / 2
    elif pol == "right":
        phase = np.pi / 2
    else:
        raise ValueError("pol must be `left` or `right`")

    # define a plane wave polarized in the y direction with a phase difference
    plane_wave_y = td.PlaneWave(
        source_time=td.GaussianPulse(freq0=freq0, fwidth=fwidth, phase=phase),
        size=(td.inf, td.inf, 0),
        center=(0, 0, 0.3 * lda0),
        direction="-",
        pol_angle=np.pi / 2,
    )

    return [plane_wave_x, plane_wave_y]


# add a flux monitor to detect transmission
monitor_r = td.FluxMonitor(
    center=[0, 0, 0.5 * lda0], size=[td.inf, td.inf, 0], freqs=freqs, name="R"
)

freq_res = td.C_0 / 2.4  # resonant frequency

# add field monitors to see the field profiles at the absorption peak frequency when GST is in the amorphous state
monitor_field_xz = td.FieldMonitor(
    center=(0, 0, 0), size=(td.inf, 0, td.inf), freqs=[freq_res], name="field_xz"
)

monitor_field_xy = td.FieldMonitor(
    center=(0, 0, 0), size=(td.inf, td.inf, 0), freqs=[freq_res], name="field_xy"
)

In [11]:
run_time = 3e-13  # simulation run time

# simulation domain box
sim_box = td.Box.from_bounds(
    rmin=(-Px / 2, -Py / 2, -t_al2o3 - t_gst - lda0 / 2), rmax=(Px / 2, Py / 2, t_al + lda0 / 2)
)


# define a function to construct a simulation given the polarization and GST state
def make_sim(pol, state):
    sim = td.Simulation(
        center=sim_box.center,
        size=sim_box.size,
        grid_spec=td.GridSpec.auto(min_steps_per_wvl=20, wavelength=lda0),
        structures=[
            left_resonator,
            right_resonator,
            buffer_layer,
            make_gst_layer(state),
            al_substrate,
        ],
        sources=circular_polarized_plane_wave(pol),
        monitors=[monitor_r, monitor_field_xz, monitor_field_xy],
        run_time=run_time,
        boundary_spec=td.BoundarySpec(
            x=td.Boundary.periodic(), y=td.Boundary.periodic(), z=td.Boundary.pml()
        ),
    )
    return sim


# make one simulation and plot it to inspect
sim_left_A = make_sim("left", "amorphous")
sim_left_A.plot_3d()

NameError: name 'GST_A' is not defined